In [ ]:
import torch
import os
from utils.data_loader import get_data_loaders
from simple_classifiers import run_simple_classifiers
from transfer_learning import run_transfer_learning
from custom_cnn import run_custom_cnn

os.makedirs("results", exist_ok=True)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Setup complete.")
print(f"Using device: {DEVICE}")

In [ ]:
train_loader, val_loader, test_loader, class_names = get_data_loaders()
num_classes = len(class_names)

In [ ]:

full_train_dataset = train_loader.dataset.dataset

from utils.metrics_plotting import show_augmentation_examples

show_augmentation_examples(dataset=full_train_dataset, class_names=class_names, num_examples=5)

In [ ]:
run_simple_classifiers(train_loader, test_loader, class_names, DEVICE)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
import numpy as np
import torch
import torchvision.models as models
import torch.nn as nn
from tqdm import tqdm

def extract_features(dataloader, model, device):
    features, labels = [], []
    model.to(device)
    model.eval()
    with torch.no_grad():
        for inputs, targets in tqdm(dataloader, desc="Extracting features"):
            inputs = inputs.to(device)
            outputs = model(inputs).cpu().numpy()
            features.append(outputs)
            labels.append(targets.numpy())
    return np.vstack(features), np.hstack(labels)

try:
    X_test.shape
    y_test.shape
    print("Używam wcześniej wyekstrahowanych cech ze zbioru testowego.")
except NameError:
    print("Cechy nie zostały znalezione. Uruchamiam ekstrakcję na nowo...")
    backbone = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT).features
    feature_extractor = nn.Sequential(backbone, nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten()).to(DEVICE)
    

    print("Ekstrahuję cechy tylko dla zbioru testowego...")
    X_test, y_test = extract_features(test_loader, feature_extractor, DEVICE)
 

num_points_to_plot = 2000
if len(X_test) > num_points_to_plot:
    print(f"\nUżywam losowej próbki {num_points_to_plot} punktów do wizualizacji t-SNE...")
    random_indices = np.random.choice(len(X_test), size=num_points_to_plot, replace=False)
    X_subset = X_test[random_indices]
    y_subset = y_test[random_indices]
else:
    X_subset = X_test
    y_subset = y_test

print("Uruchamiam algorytm t-SNE... To może potrwać kilka minut.")
tsne = TSNE(n_components=2, perplexity=30, random_state=42, max_iter=300, n_jobs=-1)
tsne_results = tsne.fit_transform(X_subset)
print("t-SNE zakończone.")


plt.figure(figsize=(14, 10))
sns.scatterplot(
    x=tsne_results[:, 0], 
    y=tsne_results[:, 1],
    hue=y_subset,
    palette=sns.color_palette("hsv", n_colors=len(np.unique(y_subset))),
    legend=False,
    alpha=0.7,
    s=50
)

plt.title('Wizualizacja Cech Obrazów za pomocą t-SNE', fontsize=16)
plt.xlabel('Komponent t-SNE 1')
plt.ylabel('Komponent t-SNE 2')
plt.grid(True)
plt.show()

In [ ]:
run_transfer_learning("MobileNetV2", num_classes, train_loader, val_loader, test_loader, class_names, DEVICE)

In [ ]:
run_transfer_learning("ResNet50", num_classes, train_loader, val_loader, test_loader, class_names, DEVICE)

In [ ]:

run_transfer_learning("EfficientNetB0", num_classes, train_loader, val_loader, test_loader, class_names, DEVICE)

In [ ]:


from custom_cnn import run_custom_cnn


BEST_LR = 0.001 

run_custom_cnn(num_classes, train_loader, val_loader, test_loader, class_names, DEVICE, lr=BEST_LR)

In [ ]:
from custom_cnn import FinalCNN, find_lr
import torch.nn as nn

test_model = FinalCNN(num_classes)
criterion = nn.CrossEntropyLoss()

find_lr(test_model, train_loader, criterion, DEVICE)

In [ ]:
from vision_transformer import run_vision_transformer

run_vision_transformer(num_classes, train_loader, val_loader, test_loader, class_names, DEVICE)